In [1]:
from datetime import datetime

import mlflow
from mlflow.data.pandas_dataset import PandasDataset
from mlflow import MlflowClient
from mlflow.entities import ViewType

import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
from pprint import pprint

path_to_analysis = Path(os.getcwd()).parent / 'Analysis'
sys.path.append(str(path_to_analysis))
from dataPrep import Config, DataPrediction

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")

In [3]:
print(str(path_to_analysis))

C:\Users\maxst\Dropbox\Weiterbildung\IU\Mods\FsME\Prog\TDSP-BsdEf\Code\Analysis


In [4]:
# Laufendes Experiment definieren
experiment_id = "AdaBooReg"

adabooreg_experiment = mlflow.set_experiment(experiment_id)

In [5]:
now = datetime.now()
now = now.strftime("%Y%m%d-%H%M")
run_name = f"AdaBooReg_{now}"

In [6]:
artifact_path = "bereitschaftsdienstplans"

In [7]:
config = Config.dict_config
pprint(config)

{'column_names_types': {'calls': 'int32',
                        'dafted': 'int16',
                        'date': 'datetime64[ns]',
                        'n_duty': 'int16',
                        'n_sby': 'int16',
                        'n_sick': 'int16',
                        'sby_need': 'int16'},
 'features': ('month',
              'dayofmonth',
              'weekday',
              'weekofyear',
              'dayofyear',
              'season'),
 'my_file_name': 'sickness_table.csv'}


In [8]:
adabooreg_run_1 = DataPrediction(**config)

Pfad zur CSV-Datei: sickness_table.csv erstellt.
Daten bereinigt und in DataFrame umgewandelt.
Daten transformiert
Features hinzugefügt


In [9]:
fitted_model = adabooreg_run_1.model_adabooreg.adabooreg_ftd

In [10]:
metrics = adabooreg_run_1.model_adabooreg.my_metrics
metrics

{'mse': 945886.8510638297, 'r2': 0.09761322118437299}

In [11]:
X_test = adabooreg_run_1.model_adabooreg.X_test
print(X_test.head())

      month  dayofmonth  weekday  weekofyear  dayofyear  season
1105      4          11        3          15        101       2
1106      4          12        4          15        102       2
1107      4          13        5          15        103       2
1108      4          14        6          15        104       2
1109      4          15        0          16        105       2


In [12]:
params = adabooreg_run_1.model_adabooreg.params_dict
pprint(params)

{'adabooreg': {'learning_rate': 0.36, 'n_estimators': 130},
 'dtreg': {'criterion': 'squared_error',
           'max_depth': 5,
           'min_samples_split': 3,
           'splitter': 'best'}}


In [26]:
print(adabooreg_run_1.df_features['date'].head())

0   2016-04-01
1   2016-04-02
2   2016-04-03
3   2016-04-04
4   2016-04-05
Name: date, dtype: datetime64[ns]


In [13]:
print(adabooreg_run_1.df_only_features.head())

   month  dayofmonth  weekday  weekofyear  dayofyear  season
0      4           1        4          13         92       2
1      4           2        5          13         93       2
2      4           3        6          13         94       2
3      4           4        0          14         95       2
4      4           5        1          14         96       2


In [14]:
target = adabooreg_run_1.df_pred['calls_reg_act_diff']
df_only_features = adabooreg_run_1.df_only_features
df_only_features['calls_reg_act_diff'] = target
df_features_target = df_only_features.copy()
print(df_features_target.head())

   month  dayofmonth  weekday  weekofyear  dayofyear  season  \
0      4           1        4          13         92       2   
1      4           2        5          13         93       2   
2      4           3        6          13         94       2   
3      4           4        0          14         95       2   
4      4           5        1          14         96       2   

   calls_reg_act_diff  
0                1095  
1                1466  
2                1026  
3                 -19  
4                 171  


In [15]:
y_train = adabooreg_run_1.model_adabooreg.adabooreg_pred_train
print(y_train)

[1407  720  529 ...  440  469  440]


In [16]:
y_test = adabooreg_run_1.model_adabooreg.adabooreg_pred
print(y_test)

[ 350  261   32 -353  285  410  377  213   66 -209 -504  271  309  213
   67  -97 -416 -624  377  394 1825 1495  980  718  559 1038  827  827
  692  567  440   45  732  707  535  431  219   25 -429  473  507  440
  271  112  -65 -456  465]


In [17]:
predictions = np.concatenate((y_train, y_test))
df_features_target['predictions'] = predictions
print(len(predictions))

1152


In [18]:
print(df_features_target.head())

   month  dayofmonth  weekday  weekofyear  dayofyear  season  \
0      4           1        4          13         92       2   
1      4           2        5          13         93       2   
2      4           3        6          13         94       2   
3      4           4        0          14         95       2   
4      4           5        1          14         96       2   

   calls_reg_act_diff  predictions  
0                1095         1407  
1                1466          720  
2                1026          529  
3                 -19         1337  
4                 171          839  


In [19]:
print(predictions)

[1407  720  529 ...  -65 -456  465]


In [20]:
dataset = mlflow.data.from_pandas(df_features_target, targets='calls_reg_act_diff', predictions='predictions')

In [21]:
# Initiate the MLflow run context
with mlflow.start_run(run_name=run_name) as run:

    # Datensatz protokollieren
    mlflow.log_input(dataset)
    
    # Parameter protokollieren
    mlflow.log_params(params)

    # Metriks protokollieren
    mlflow.log_metrics(metrics)

    # trainiertes Modell-Instanz protokollieren
    mlflow.sklearn.log_model(
        sk_model=fitted_model, 
        input_example=X_test,
        artifact_path=artifact_path
    )

mlflow.end_run()

C:\Users\maxst\Dropbox\Weiterbildung\IU\Mods\FsME\Prog\TDSP-BsdEf\BsdEf_venv\lib\site-packages\mlflow\data\pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  return _infer_schema(self._df)
C:\Users\maxst\Dropbox\Weiterbildung\IU\Mods\FsME\Prog\TDSP-BsdEf\BsdEf_venv\lib\site-packages\mlflow\models\signature.py:351: UserWarning: Hint: Inferred schema contain

In [22]:
query = "attribute.name = 'AdaBooReg'"
exp = mlflow.search_experiments(filter_string=query)
pprint(exp)

[<Experiment: artifact_location='mlflow-artifacts:/434640519357902442', creation_time=1700060513796, experiment_id='434640519357902442', last_update_time=1700060513796, lifecycle_stage='active', name='AdaBooReg', tags={'jahr': '2019',
 'mlflow.note.content': 'AdaBoostRegressor-Vorhersage für Notrufe minus '
                        'Regressionsausgabe.',
 'month': 'Mai',
 'project_name': 'Vorhersagemodell für Einsatzfahrende'}>]


In [23]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/429532028009648294', creation_time=1700657534185, experiment_id='429532028009648294', last_update_time=1700657534185, lifecycle_stage='active', name='VorhersageModell', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/162309136132002052', creation_time=1700644939220, experiment_id='162309136132002052', last_update_time=1700644939220, lifecycle_stage='active', name='TrendRegression', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/434640519357902442', creation_time=1700060513796, experiment_id='434640519357902442', last_update_time=1700060513796, lifecycle_stage='active', name='AdaBooReg', tags={'jahr': '2019',
  'mlflow.note.content': 'AdaBoostRegressor-Vorhersage für Notrufe minus '
                         'Regressionsausgabe.',
  'month': 'Mai',
  'project_name': 'Vorhersagemodell für Einsatzfahrende'}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1700055785031, experiment_id='0', la

In [24]:
runs = mlflow.search_runs(experiment_names=["AdaBooReg"])
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.mse,metrics.r2,params.adabooreg,params.dtreg,...,params.est__splitter,params.est__min_samples_split,params.learning_rate,params.n_estimators,params.est__max_depth,tags.mlflow.log-model.history,tags.mlflow.runName,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.source.name
0,4ef0362b718d4d2ba2ec3314577ea449,434640519357902442,FINISHED,mlflow-artifacts:/434640519357902442/4ef0362b7...,2023-11-22 13:10:31.996000+00:00,2023-11-22 13:10:37.447000+00:00,945886.851064,0.097613,"{'n_estimators': 130, 'learning_rate': 0.36}","{'criterion': 'squared_error', 'splitter': 'be...",...,None,None,None,None,None,"[{""run_id"": ""4ef0362b718d4d2ba2ec3314577ea449""...",AdaBooReg_20231122-1410,maxst,LOCAL,C:\Users\maxst\Dropbox\Weiterbildung\IU\Mods\F...
1,166ae2280b4049b993fa2f15d9719ca0,434640519357902442,FINISHED,mlflow-artifacts:/434640519357902442/166ae2280...,2023-11-22 12:57:27.556000+00:00,2023-11-22 12:57:32.809000+00:00,945886.851064,0.097613,"{'n_estimators': 130, 'learning_rate': 0.36}","{'criterion': 'squared_error', 'splitter': 'be...",...,None,None,None,None,None,"[{""run_id"": ""166ae2280b4049b993fa2f15d9719ca0""...",AdaBooReg_20231122-1357,maxst,LOCAL,C:\Users\maxst\Dropbox\Weiterbildung\IU\Mods\F...
2,be15aac7b77e405cbd4a6c6489570596,434640519357902442,FINISHED,mlflow-artifacts:/434640519357902442/be15aac7b...,2023-11-22 12:53:29.119000+00:00,2023-11-22 12:53:35.197000+00:00,945886.851064,0.097613,"{'n_estimators': 130, 'learning_rate': 0.36}","{'criterion': 'squared_error', 'splitter': 'be...",...,None,None,None,None,None,"[{""run_id"": ""be15aac7b77e405cbd4a6c6489570596""...",AdaBooReg_20231122-1353,maxst,LOCAL,C:\Users\maxst\Dropbox\Weiterbildung\IU\Mods\F...
3,2b28bf0350fb4428a3a644d344404ee3,434640519357902442,FINISHED,mlflow-artifacts:/434640519357902442/2b28bf035...,2023-11-22 10:04:03.559000+00:00,2023-11-22 10:04:08.874000+00:00,945886.851064,0.097613,"{'n_estimators': 130, 'learning_rate': 0.36}","{'criterion': 'squared_error', 'splitter': 'be...",...,None,None,None,None,None,"[{""run_id"": ""2b28bf0350fb4428a3a644d344404ee3""...",AdaBooReg_20231122-1104,maxst,LOCAL,C:\Users\maxst\Dropbox\Weiterbildung\IU\Mods\F...
4,8d8a6404da1a4493b371fad742bcadf9,434640519357902442,FINISHED,mlflow-artifacts:/434640519357902442/8d8a6404d...,2023-11-22 09:42:41.902000+00:00,2023-11-22 09:42:51.825000+00:00,945886.851064,0.097613,"{'n_estimators': 130, 'learning_rate': 0.36}","{'criterion': 'squared_error', 'splitter': 'be...",...,None,None,None,None,None,"[{""run_id"": ""8d8a6404da1a4493b371fad742bcadf9""...",AdaBooReg_1,maxst,LOCAL,C:\Users\maxst\Dropbox\Weiterbildung\IU\Mods\F...
5,1e01351760c44b7a8b073329397e8d08,434640519357902442,FINISHED,mlflow-artifacts:/434640519357902442/1e0135176...,2023-11-22 09:36:01.868000+00:00,2023-11-22 09:36:13.941000+00:00,945886.851064,0.097613,"{'n_estimators': 130, 'learning_rate': 0.36}","{'criterion': 'squared_error', 'splitter': 'be...",...,None,None,None,None,None,"[{""run_id"": ""1e01351760c44b7a8b073329397e8d08""...",AdaBooReg_1,maxst,LOCAL,C:\Users\maxst\Dropbox\Weiterbildung\IU\Mods\F...
6,b13bee8fa89a423386baccc8c86ecf5e,434640519357902442,FINISHED,mlflow-artifacts:/434640519357902442/b13bee8fa...,2023-11-22 09:20:51.059000+00:00,2023-11-22 09:21:03.843000+00:00,945886.851064,0.097613,"{'n_estimators': 130, 'learning_rate': 0.36}","{'criterion': 'squared_error', 'splitter': 'be...",...,None,None,None,None,None,"[{""run_id"": ""b13bee8fa89a423386baccc8c86ecf5e""...",AdaBooReg_1,maxst,LOCAL,C:\Users\maxst\Dropbox\Weiterbildung\IU\Mods\F...
7,2374698eb8374f6ca6a0411aef60f465,434640519357902442,FINISHED,mlflow-artifacts:/434640519357902442/2374698eb...,2023-11-21 17:19:41.706000+00:00,2023-11-21 17:19:51.822000+00:00,945886.851064,0.097613,"{'n_estimators': 130, 'learning_rate': 0.36}","{'criterion': 'squared_error', 'splitter': 'be...",...,None,None,None,None,None,"[{""run_id"": ""2374698eb8374f6ca6a0411aef60f465""...